## GAN 50 sqr

This notebook is modified from the keras repository: https://github.com/BAI-Yeqi/Keras-GAN/blob/master/gan/gan.py

As tested (256, 256) is too big a input, so we are using (50, 50)


In [4]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [2]:
X_try = np.load("drive/Colab Notebooks/Data/ChestX_8/50sqr/image_array_batch_0.npy")
X_try.shape

(2000, 50, 50)

The Training Set of MNIST got the shape 

```
(60000, 28, 28)
```



In [5]:
class GAN():
    def __init__(self):
        self.img_rows = 50 
        self.img_cols = 50
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', 
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(400,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity 
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (400,)
        
        model = Sequential()

        model.add(Dense(1024, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Dense(2000, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Dense(4000))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Dense(2500))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)
        
        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(3000))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1500))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        # (X_train, _), (_, _) = mnist.load_data()
        X_train = np.load("drive/Colab Notebooks/Data/ChestX_8/50sqr/image_array_batch_0.npy")
        
        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 400))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 400))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)
                
            if epoch % 5000 == 0:
                self.generator.save("drive/Colab Notebooks/GAN Medical Image/Models/Fir_GAN_G_epoch_"+ str(epoch) +".h5")
                self.discriminator.save("drive/Colab Notebooks/GAN Medical Image/Models/Fir_GAN_D_epoch_"+ str(epoch) +".h5")
    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 400))
        print("shape of noise: "+ str(noise.shape))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("drive/Colab Notebooks/GAN Medical Image/Images Gen/Images GAN FirApt2/FirGAN_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, save_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 2500)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3000)              7503000   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 3000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1500)              4501500   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1500)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               384256    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 256)               0         
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.768200, acc.: 21.88%] [G loss: 0.340165]
shape of noise: (25, 400)
1 [D loss: 0.381696, acc.: 75.00%] [G loss: 1.692652]
2 [D loss: 0.025548, acc.: 100.00%] [G loss: 3.851747]
3 [D loss: 0.013639, acc.: 100.00%] [G loss: 4.597534]
4 [D loss: 0.007388, acc.: 100.00%] [G loss: 4.988190]
5 [D loss: 0.070329, acc.: 96.88%] [G loss: 4.484633]
6 [D loss: 0.016084, acc.: 100.00%] [G loss: 4.958931]
7 [D loss: 0.012700, acc.: 100.00%] [G loss: 5.747693]
8 [D loss: 0.044855, acc.: 96.88%] [G loss: 5.465163]
9 [D loss: 0.020388, acc.: 100.00%] [G loss: 5.805025]
10 [D loss: 0.002335, acc.: 100.00%] [G loss: 6.060400]
11 [D loss: 0.003912, acc.: 100.00%] [G loss: 6.416429]
12 [D loss: 0.028294, acc.: 100.00%] [G loss: 7.479999]
13 [D loss: 0.000866, acc.: 100.00%] [G loss: 8.258133]
14 [D loss: 0.004641, acc.: 100.00%] [G loss: 8.331898]
15 [D loss: 0.000744, acc.: 100.00%] [G loss: 7.915349]
16 [D loss: 0.009125, acc.: 100.00%] [G loss: 8.296771]
17 [D loss: 0.045633, acc.: 96.88%] 

101 [D loss: 0.358332, acc.: 87.50%] [G loss: 8.030148]
102 [D loss: 0.157618, acc.: 100.00%] [G loss: 8.744620]
103 [D loss: 0.358116, acc.: 75.00%] [G loss: 7.403963]
104 [D loss: 0.378841, acc.: 84.38%] [G loss: 8.866526]
105 [D loss: 0.092562, acc.: 96.88%] [G loss: 8.154781]
106 [D loss: 0.204568, acc.: 90.62%] [G loss: 7.569684]
107 [D loss: 0.093524, acc.: 100.00%] [G loss: 8.222876]
108 [D loss: 0.386713, acc.: 71.88%] [G loss: 12.924949]
109 [D loss: 1.204867, acc.: 43.75%] [G loss: 11.295628]
110 [D loss: 0.211320, acc.: 90.62%] [G loss: 10.317575]
111 [D loss: 0.526374, acc.: 78.12%] [G loss: 8.079993]
112 [D loss: 0.243464, acc.: 87.50%] [G loss: 9.098168]
113 [D loss: 0.291764, acc.: 93.75%] [G loss: 6.961831]
114 [D loss: 0.113920, acc.: 96.88%] [G loss: 7.125813]
115 [D loss: 0.135488, acc.: 96.88%] [G loss: 7.139155]
116 [D loss: 0.301627, acc.: 81.25%] [G loss: 7.113485]
117 [D loss: 0.175055, acc.: 96.88%] [G loss: 6.350100]
118 [D loss: 0.287706, acc.: 90.62%] [G los

200 [D loss: 0.143210, acc.: 100.00%] [G loss: 7.164383]
shape of noise: (25, 400)
201 [D loss: 0.673600, acc.: 75.00%] [G loss: 9.672514]
202 [D loss: 0.450757, acc.: 81.25%] [G loss: 7.749704]
203 [D loss: 0.302810, acc.: 90.62%] [G loss: 7.506021]
204 [D loss: 0.173166, acc.: 90.62%] [G loss: 7.373043]
205 [D loss: 0.760164, acc.: 62.50%] [G loss: 8.303358]
206 [D loss: 0.491764, acc.: 68.75%] [G loss: 8.386949]
207 [D loss: 0.233256, acc.: 93.75%] [G loss: 7.118296]
208 [D loss: 0.370798, acc.: 84.38%] [G loss: 8.662592]
209 [D loss: 0.105815, acc.: 96.88%] [G loss: 8.172719]
210 [D loss: 0.340792, acc.: 87.50%] [G loss: 7.323090]
211 [D loss: 0.216404, acc.: 93.75%] [G loss: 9.053085]
212 [D loss: 0.268122, acc.: 84.38%] [G loss: 8.533819]
213 [D loss: 0.509440, acc.: 81.25%] [G loss: 12.113899]
214 [D loss: 0.610608, acc.: 65.62%] [G loss: 8.844331]
215 [D loss: 0.293900, acc.: 84.38%] [G loss: 7.763460]
216 [D loss: 0.323751, acc.: 87.50%] [G loss: 7.170133]
217 [D loss: 1.33384

...

29551 [D loss: 0.023637, acc.: 100.00%] [G loss: 2.011379]
29552 [D loss: 0.012761, acc.: 100.00%] [G loss: 4.040542]
29553 [D loss: 0.027928, acc.: 100.00%] [G loss: 2.038671]
29554 [D loss: 0.018696, acc.: 100.00%] [G loss: 2.225745]
29555 [D loss: 0.224960, acc.: 87.50%] [G loss: 1.008555]
29556 [D loss: 0.016617, acc.: 100.00%] [G loss: 4.980798]
29557 [D loss: 0.004570, acc.: 100.00%] [G loss: 2.915671]
29558 [D loss: 0.081101, acc.: 96.88%] [G loss: 1.113734]
29559 [D loss: 0.088564, acc.: 93.75%] [G loss: 1.601696]
29560 [D loss: 0.052846, acc.: 96.88%] [G loss: 1.055968]
29561 [D loss: 0.019025, acc.: 100.00%] [G loss: 1.576374]
29562 [D loss: 0.103506, acc.: 96.88%] [G loss: 2.256936]
29563 [D loss: 0.006403, acc.: 100.00%] [G loss: 1.805496]
29564 [D loss: 0.029980, acc.: 96.88%] [G loss: 3.186955]
29565 [D loss: 0.006116, acc.: 100.00%] [G loss: 7.742770]
29566 [D loss: 0.024677, acc.: 100.00%] [G loss: 1.138846]
29567 [D loss: 0.007752, acc.: 100.00%] [G loss: 2.641498]
295

29648 [D loss: 0.163307, acc.: 87.50%] [G loss: 1.560481]
29649 [D loss: 0.009968, acc.: 100.00%] [G loss: 3.267490]
29650 [D loss: 0.207314, acc.: 93.75%] [G loss: 1.273224]
29651 [D loss: 0.007720, acc.: 100.00%] [G loss: 2.835732]
29652 [D loss: 0.011891, acc.: 100.00%] [G loss: 0.664241]
29653 [D loss: 0.179256, acc.: 96.88%] [G loss: 2.397478]
29654 [D loss: 0.002491, acc.: 100.00%] [G loss: 1.242763]
29655 [D loss: 0.008189, acc.: 100.00%] [G loss: 4.037785]
29656 [D loss: 0.021376, acc.: 100.00%] [G loss: 1.258269]
29657 [D loss: 0.023181, acc.: 100.00%] [G loss: 7.973149]
29658 [D loss: 0.014191, acc.: 100.00%] [G loss: 1.310413]
29659 [D loss: 0.004500, acc.: 100.00%] [G loss: 0.918862]
29660 [D loss: 0.000754, acc.: 100.00%] [G loss: 15.212072]
29661 [D loss: 0.003460, acc.: 100.00%] [G loss: 1.052892]
29662 [D loss: 0.004310, acc.: 100.00%] [G loss: 9.252596]
29663 [D loss: 0.036069, acc.: 96.88%] [G loss: 4.905282]
29664 [D loss: 0.002160, acc.: 100.00%] [G loss: 1.151911]


29747 [D loss: 0.382860, acc.: 93.75%] [G loss: 2.143916]
29748 [D loss: 0.007129, acc.: 100.00%] [G loss: 1.201296]
29749 [D loss: 0.194345, acc.: 96.88%] [G loss: 5.198633]
29750 [D loss: 0.032273, acc.: 96.88%] [G loss: 2.218755]
29751 [D loss: 0.083746, acc.: 96.88%] [G loss: 1.040537]
29752 [D loss: 0.038165, acc.: 100.00%] [G loss: 6.633182]
29753 [D loss: 0.004223, acc.: 100.00%] [G loss: 2.135890]
29754 [D loss: 0.118564, acc.: 96.88%] [G loss: 1.504583]
29755 [D loss: 0.010690, acc.: 100.00%] [G loss: 0.829181]
29756 [D loss: 0.005001, acc.: 100.00%] [G loss: 0.961658]
29757 [D loss: 0.008074, acc.: 100.00%] [G loss: 2.082199]
29758 [D loss: 0.043979, acc.: 96.88%] [G loss: 0.984319]
29759 [D loss: 0.180127, acc.: 90.62%] [G loss: 1.140664]
29760 [D loss: 0.071492, acc.: 96.88%] [G loss: 1.020144]
29761 [D loss: 0.072948, acc.: 96.88%] [G loss: 1.299073]
29762 [D loss: 0.040371, acc.: 96.88%] [G loss: 1.000603]
29763 [D loss: 0.001426, acc.: 100.00%] [G loss: 1.045763]
29764 [

29846 [D loss: 0.225958, acc.: 93.75%] [G loss: 3.932261]
29847 [D loss: 0.060898, acc.: 96.88%] [G loss: 1.474591]
29848 [D loss: 0.193569, acc.: 90.62%] [G loss: 3.395444]
29849 [D loss: 0.137459, acc.: 96.88%] [G loss: 2.084302]
29850 [D loss: 0.344519, acc.: 81.25%] [G loss: 2.718067]
29851 [D loss: 0.293036, acc.: 90.62%] [G loss: 5.399427]
29852 [D loss: 0.502655, acc.: 84.38%] [G loss: 3.762187]
29853 [D loss: 0.477365, acc.: 84.38%] [G loss: 3.626931]
29854 [D loss: 0.230952, acc.: 90.62%] [G loss: 4.404275]
29855 [D loss: 0.257081, acc.: 84.38%] [G loss: 2.335055]
29856 [D loss: 0.207666, acc.: 96.88%] [G loss: 5.390154]
29857 [D loss: 0.019724, acc.: 100.00%] [G loss: 3.840532]
29858 [D loss: 0.046112, acc.: 100.00%] [G loss: 3.148751]
29859 [D loss: 0.385274, acc.: 87.50%] [G loss: 3.905013]
29860 [D loss: 0.042200, acc.: 100.00%] [G loss: 3.132955]
29861 [D loss: 0.324936, acc.: 87.50%] [G loss: 2.624746]
29862 [D loss: 0.127251, acc.: 93.75%] [G loss: 2.595978]
29863 [D lo

29945 [D loss: 0.005455, acc.: 100.00%] [G loss: 3.706707]
29946 [D loss: 0.064380, acc.: 100.00%] [G loss: 2.467477]
29947 [D loss: 0.063200, acc.: 96.88%] [G loss: 1.004003]
29948 [D loss: 0.049836, acc.: 100.00%] [G loss: 1.130757]
29949 [D loss: 0.017744, acc.: 100.00%] [G loss: 3.193427]
29950 [D loss: 0.037057, acc.: 96.88%] [G loss: 1.279291]
29951 [D loss: 0.045012, acc.: 96.88%] [G loss: 2.243129]
29952 [D loss: 0.081812, acc.: 96.88%] [G loss: 1.424006]
29953 [D loss: 0.040207, acc.: 96.88%] [G loss: 1.825703]
29954 [D loss: 0.001330, acc.: 100.00%] [G loss: 2.168945]
29955 [D loss: 0.025864, acc.: 100.00%] [G loss: 1.043939]
29956 [D loss: 0.056657, acc.: 100.00%] [G loss: 2.427041]
29957 [D loss: 0.061125, acc.: 96.88%] [G loss: 3.064876]
29958 [D loss: 0.007826, acc.: 100.00%] [G loss: 3.068321]
29959 [D loss: 0.063663, acc.: 96.88%] [G loss: 1.131503]
29960 [D loss: 0.063345, acc.: 96.88%] [G loss: 2.834191]
29961 [D loss: 0.077331, acc.: 96.88%] [G loss: 1.773614]
29962 

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install -q keras

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2177346558390061861, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11297803469
 locality {
   bus_id: 1
 }
 incarnation: 6388064711824174890
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]